In [1]:
import pandas as pd


def load_rxnconso(file_path):
    """
    Load the RXNCONSO.RRF file into a pandas DataFrame.

    Args:
    file_path (str): The path to the RXNCONSO.RRF file.

    Returns:
    pd.DataFrame: A DataFrame containing the RXNCONSO data.
    """
    column_names = [
        "RXCUI",
        "LAT",
        "TS",
        "LUI",
        "STT",
        "SUI",
        "ISPREF",
        "RXAUI",
        "SAUI",
        "SCUI",
        "SDUI",
        "SAB",
        "TTY",
        "CODE",
        "STR",
        "SRL",
        "SUPPRESS",
        "CVF",
    ]
    df = pd.read_csv(file_path, sep="|", names=column_names, index_col=False)
    return df


def find_max_rxcui(df):
    """
    Find the maximum RxCUI value in the DataFrame.

    Args:
    df (pd.DataFrame): The DataFrame containing RXNCONSO data.

    Returns:
    int: The maximum RxCUI value.
    """
    df["RXCUI"] = pd.to_numeric(df["RXCUI"], errors="coerce")
    max_rxcui = df["RXCUI"].max()
    return max_rxcui


def pair_brand_generic_names(df):
    """
    Pair brand and generic names from the RXNCONSO DataFrame.

    Args:
    df (pd.DataFrame): The DataFrame containing RXNCONSO data.

    Returns:
    pd.DataFrame: A DataFrame with paired brand and generic names.
    """
    # Filter out relevant TTY (Term Type) for brand and generic names
    brand_names = df[df["TTY"] == "BN"]  # BN: Brand Name
    generic_names = df[df["TTY"] == "IN"]  # IN: Ingredient Name

    # Normalize strings for comparison
    generic_names["STR_normalized"] = generic_names["STR"].str.strip().str.lower()

    # Ensure unique generic names per RxCUI
    generic_names = generic_names.drop_duplicates(subset=["RXCUI"])

    # Merge brand and generic names on RXCUI
    paired_names = pd.merge(
        brand_names, generic_names, on="RXCUI", suffixes=("_brand", "_generic")
    )
    paired_names = paired_names[["RXCUI", "STR_brand", "STR_generic"]]
    paired_names.columns = ["RxCUI", "BrandName", "GenericName"]

    return paired_names

In [2]:
# Load the data
file_path = "data/RXNCONSO.RRF"
rxnconso_df = load_rxnconso(file_path)

rxnconso_df.head()

/tmp/ipykernel_22159/945484680.py:34: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep="|", names=column_names, index_col=False)


,RXCUI,LAT,TS,LUI,STT,SUI,ISPREF,RXAUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,3,ENG,NaN,NaN,NaN,NaN,NaN,8717795,NaN,58488005,NaN,SNOMEDCT_US,PT,58488005,"1,4-alpha-Glucan branching enzyme",NaN,N,NaN
1,3,ENG,NaN,NaN,NaN,NaN,NaN,8717796,NaN,58488005,NaN,SNOMEDCT_US,FN,58488005,"1,4-alpha-Glucan branching enzyme (substance)",NaN,N,NaN
2,3,ENG,NaN,NaN,NaN,NaN,NaN,8717808,NaN,58488005,NaN,SNOMEDCT_US,SY,58488005,"Amylo-(1,4,6)-transglycosylase",NaN,N,NaN
3,3,ENG,NaN,NaN,NaN,NaN,NaN,8718164,NaN,58488005,NaN,SNOMEDCT_US,SY,58488005,Branching enzyme,NaN,N,NaN
4,19,ENG,NaN,NaN,NaN,NaN,NaN,10794494,NaN,112116001,NaN,SNOMEDCT_US,SY,112116001,17-hydrocorticosteroid,NaN,N,NaN


In [3]:
# Find the maximum RxCUI value
max_rxcui = find_max_rxcui(rxnconso_df)
print("Maximum RxCUI value:", max_rxcui)

Maximum RxCUI value: 2679989


In [4]:
# Pair brand and generic names
paired_names = pair_brand_generic_names(rxnconso_df)
print(paired_names.head())

   RxCUI                  BrandName                GenericName
0     44                      Mesna                      Mesna
1     74                       PABA          Aminobenzoic Acid
2     94                      5-HTP                 Oxitriptan
3     97  Ticlopidine Hydrochloride  Ticlopidine Hydrochloride
4     99          Aminocaproic Acid          Aminocaproic Acid


/tmp/ipykernel_22159/945484680.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  generic_names["STR_normalized"] = generic_names["STR"].str.strip().str.lower()


In [5]:
# Check if any RxCUI has more than one unique generic name
for rxcui, group in paired_names.groupby("RxCUI"):
    if len(group["GenericName"].unique()) > 1:
        print(f"RxCUI {rxcui} has multiple generic names:")
        print(group)
        print()

In [6]:
# Check if any RxCUI has more than one unique brand name
for rxcui, group in paired_names.groupby("RxCUI"):
    if len(group["BrandName"].unique()) > 1:
        print(f"RxCUI {rxcui} has multiple brand names:")
        print(group)
        print()

RxCUI 161 has multiple brand names:
   RxCUI        BrandName    GenericName
6    161  APAP (obsolete)  Acetaminophen
7    161    Acetaminophen  Acetaminophen
8    161      Paracetamol  Acetaminophen

RxCUI 168 has multiple brand names:
    RxCUI            BrandName  GenericName
10    168     Acetic Acid Otic  Acetic Acid
11    168          Acetic Acid  Acetic Acid
12    168  Acetic Acid Glacial  Acetic Acid

RxCUI 272 has multiple brand names:
    RxCUI           BrandName         GenericName
16    272       Active Carbon  Activated Charcoal
17    272  Activated Charcoal  Activated Charcoal

RxCUI 448 has multiple brand names:
    RxCUI      BrandName GenericName
26    448  Ethyl Alcohol     Alcohol
27    448        Alcohol     Alcohol

RxCUI 632 has multiple brand names:
    RxCUI    BrandName GenericName
34    632  Mitomycin-C   Mitomycin
35    632    MitoMYcin   Mitomycin

RxCUI 1091 has multiple brand names:
    RxCUI   BrandName GenericName
47   1091    Arginine    Arginine
48  

In [7]:
# number of unique generic names
print(len(paired_names["GenericName"].unique()))

# number of unique brand names
print(len(paired_names["BrandName"].unique()))

1896
2074


In [8]:
# Rename the columns rxcui,generic,brand
paired_names.columns = ["rxcui", "generic", "brand"]

# Save the paired names to a CSV file
paired_names.to_csv("paired_brand_generic_names.csv", index=False)